In [1]:
%load_ext dotenv

%dotenv ../tests/.env.test
%dotenv -o ../.env


In [2]:
from uuid import uuid4
import json

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.documents import Document

import redbox
from redbox.app import Redbox
from redbox.models.chain import RedboxState


In [3]:

app = Redbox(debug=False)

In [4]:
from langfuse.openai import AsyncAzureOpenAI

In [5]:
s = RedboxState(
    user_uuid="5210311c-df38-430f-8451-df5fa28f0be3", # Get this from the URL in the admin console while viewing your local superuser
    messages=[
            HumanMessage("What is this report?"),
            AIMessage("The report is Tony_Blair_Institute__Governing_in_the_Age_of_AI__May_2024"),
            HumanMessage("What is it about?")
        ],
    documents=[
        Document("We should create a new AI team in 10 Downing Street", metadata={"uri": "Tony_Blair_Institute__Governing_in_the_Age_of_AI__May_2024"})
    ]
)

# s = RedboxState(
#     user_uuid="5210311c-df38-430f-8451-df5fa28f0be3", # Get this from the URL in the admin console while viewing your local superuser
#     messages=[
#             HumanMessage("What is the capital of France?"),
#             AIMessage("The capital of France is Paris"),
#             HumanMessage("How do you say that in French?")
#         ],
#     documents=[]
# )


In [ ]:

response_text = ""
async def token_handler(s):
    global response_text
    response_text += s

r = app.run_sync(
    input=s
)

# r = await app.run(
#     input=s,
#     response_tokens_callback=token_handler,
# )

print("*"*10)
print()
print(f"User: {s.messages[-1]}")
print()
print(f"AI: {response_text}")
print()


In [ ]:
r.content